In [2]:
#!/usr/bin/env python
# coding: utf-8

# # Pronóstico de Casos de Dengue por Barrio usando GRU
# ## Red Neuronal Recurrente para Series Temporales con lluvia y temperatura

# ### 1. Configuración e Importación de Librerías

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import optuna
import plotly.graph_objects as go
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configurar dispositivo de cómputo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Dispositivo de cómputo: {device}")

# Establecer semilla para reproducibilidad
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

# ### 2. Carga y Preparación de Datos

# Cargar datos desde el archivo Parquet
df = pd.read_parquet('../../Datos/df_train.parquet')
print(f"Dimensiones del DataFrame: {df.shape}")
print("\nPrimeras filas:")
print(df.head())

# Crear columna de fecha usando formato de semanas ISO
df['fecha'] = pd.to_datetime(df['anio'].astype(str) + df['semana'].astype(str) + '1', format='%G%V%u')

# Establecer fecha como índice y ordenar
df = df.set_index('fecha').sort_index()
print(f"\nRango temporal: {df.index.min()} - {df.index.max()}")

# ### 3. Preprocesamiento e Ingeniería de Características

# División de datos según el año
train_data = df[df['anio'] < 2021]
val_data = df[df['anio'] == 2021]
print(f"\nTamaño conjunto entrenamiento: {len(train_data)}")
print(f"Tamaño conjunto validación: {len(val_data)}")

# Definir columnas a usar
feature_cols = ['lluvia_mean', 'temperatura_mean']
target_col = 'dengue'

print(f"\nColumnas de características: {feature_cols}")
print(f"Variable objetivo: {target_col}")

# Escalar variables
scaler_features = StandardScaler()
scaler_target = StandardScaler()

# Ajustar escaladores con datos de entrenamiento
scaler_features.fit(train_data[feature_cols])
scaler_target.fit(train_data[[target_col]])

# Aplicar escalado
train_scaled = train_data.copy()
val_scaled = val_data.copy()
train_scaled[feature_cols] = scaler_features.transform(train_data[feature_cols])
train_scaled[target_col] = scaler_target.transform(train_data[[target_col]])
val_scaled[feature_cols] = scaler_features.transform(val_data[feature_cols])
val_scaled[target_col] = scaler_target.transform(val_data[[target_col]])

# ### Función para crear secuencias

def create_sequences(data, window_size, target_col, feature_cols):
    """
    Crea secuencias de ventanas deslizantes para el modelo GRU.
    
    Args:
        data: DataFrame con los datos
        window_size: Tamaño de la ventana de entrada
        target_col: Nombre de la columna objetivo
        feature_cols: Lista de columnas de características
    
    Returns:
        sequences: Array con las secuencias
        targets: Array con los valores objetivo
        barrio_ids: Array con los IDs de barrio para cada secuencia
    """
    sequences = []
    targets = []
    barrio_ids = []
    
    # Agrupar por barrio para mantener continuidad temporal
    for barrio in data['id_bar'].unique():
        barrio_data = data[data['id_bar'] == barrio].sort_index()
        
        if len(barrio_data) <= window_size:
            continue
            
        for i in range(len(barrio_data) - window_size):
            # Secuencia de características (lluvia y temperatura)
            seq = barrio_data[feature_cols].iloc[i:i+window_size].values
            # Valor objetivo (dengue)
            target = barrio_data[target_col].iloc[i+window_size]
            
            sequences.append(seq)
            targets.append(target)
            barrio_ids.append(barrio)
    
    return (np.array(sequences, dtype=np.float32), 
            np.array(targets, dtype=np.float32),
            np.array(barrio_ids))

# ### 4. Definición del Modelo GRU con PyTorch

class DengueDataset(Dataset):
    """Dataset personalizado para las secuencias de dengue"""
    def __init__(self, sequences, targets):
        self.sequences = sequences
        self.targets = targets
    
    def __len__(self):
        return len(self.targets)
    
    def __getitem__(self, idx):
        return (torch.FloatTensor(self.sequences[idx]),
                torch.FloatTensor([self.targets[idx]]))

class GRUModel(nn.Module):
    """
    Modelo GRU simple para series temporales
    """
    def __init__(self, input_size, hidden_size, num_layers, dropout_rate):
        super(GRUModel, self).__init__()
        
        # Capas GRU
        self.gru = nn.GRU(input_size, hidden_size, num_layers, 
                         batch_first=True, dropout=dropout_rate if num_layers > 1 else 0)
        
        # Dropout
        self.dropout = nn.Dropout(dropout_rate)
        
        # Capa de salida
        self.fc = nn.Linear(hidden_size, 1)
        
    def forward(self, x):
        # Pasar por GRU
        out, _ = self.gru(x)
        
        # Tomar la salida del último paso temporal
        out = out[:, -1, :]
        out = self.dropout(out)
        out = self.fc(out)
        
        return out

# ### 5. Optimización de Hiperparámetros con Optuna

def objective(trial):
    """
    Función objetivo para Optuna
    """
    # Espacio de búsqueda de hiperparámetros
    window_size = trial.suggest_int('window_size', 8, 48)
    hidden_size = trial.suggest_categorical('hidden_size', [16, 32, 64, 128])
    num_layers = trial.suggest_int('num_layers', 1, 3)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])
    
    # Crear secuencias
    X_train, y_train, _ = create_sequences(
        train_scaled, window_size, target_col, feature_cols
    )
    X_val, y_val, _ = create_sequences(
        val_scaled, window_size, target_col, feature_cols
    )
    
    # Crear datasets y dataloaders
    train_dataset = DengueDataset(X_train, y_train)
    val_dataset = DengueDataset(X_val, y_val)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    # Configurar modelo
    input_size = len(feature_cols)  # 2 (lluvia_mean, temperatura_mean)
    
    model = GRUModel(input_size, hidden_size, num_layers, dropout_rate).to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    # Entrenamiento
    n_epochs = 40
    best_val_loss = float('inf')
    patience = 20
    patience_counter = 0
    
    for epoch in range(n_epochs):
        # Training
        model.train()
        train_loss = 0
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            
            optimizer.zero_grad()
            output = model(x)
            loss = criterion(output, y)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
        
        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device), y.to(device)
                output = model(x)
                loss = criterion(output, y)
                val_loss += loss.item()
        
        val_loss /= len(val_loader)
        
        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1
            
        if patience_counter >= patience:
            break
    
    return best_val_loss

# Ejecutar optimización con Optuna
print("\n### Iniciando optimización de hiperparámetros con Optuna ###")
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=40, show_progress_bar=True)

print("\nMejores hiperparámetros encontrados:")
print(study.best_params)
print(f"\nMejor MSE en validación: {study.best_value:.4f}")

# ### 6. Entrenamiento del Modelo Final

# Obtener mejores hiperparámetros
best_params = study.best_params
window_size = best_params['window_size']
hidden_size = best_params['hidden_size']
num_layers = best_params['num_layers']
dropout_rate = best_params['dropout_rate']
learning_rate = best_params['learning_rate']
batch_size = best_params['batch_size']

print("\n### Entrenando modelo final con datos combinados ###")

# Combinar datos de entrenamiento y validación
combined_data = pd.concat([train_data, val_data])
combined_scaled = combined_data.copy()
combined_scaled[feature_cols] = scaler_features.transform(combined_data[feature_cols])
combined_scaled[target_col] = scaler_target.transform(combined_data[[target_col]])

# Crear secuencias con datos combinados
X_combined, y_combined, _ = create_sequences(
    combined_scaled, window_size, target_col, feature_cols
)

# Crear dataset y dataloader
combined_dataset = DengueDataset(X_combined, y_combined)
combined_loader = DataLoader(combined_dataset, batch_size=batch_size, shuffle=True)

# Configurar modelo final
input_size = len(feature_cols)
model_final = GRUModel(input_size, hidden_size, num_layers, dropout_rate).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model_final.parameters(), lr=learning_rate)

# Entrenamiento del modelo final
n_epochs = 40
train_losses = []

for epoch in range(n_epochs):
    model_final.train()
    epoch_loss = 0
    
    for x, y in combined_loader:
        x, y = x.to(device), y.to(device)
        
        optimizer.zero_grad()
        output = model_final(x)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    
    avg_loss = epoch_loss / len(combined_loader)
    train_losses.append(avg_loss)
    
    if (epoch + 1) % 10 == 0:
        print(f"Época {epoch+1}/{n_epochs}, Pérdida: {avg_loss:.4f}")

# Visualizar curva de pérdida
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(1, n_epochs+1)), y=train_losses,
                        mode='lines', name='Pérdida de Entrenamiento'))
fig.update_layout(title='Curva de Pérdida durante el Entrenamiento',
                  xaxis_title='Época', yaxis_title='MSE')
fig.show()

# ### 7. Generación de Pronósticos para 2022

print("\n### Generando pronósticos para 2022 ###")

# Preparar datos para pronóstico
all_data = df.copy()
all_data_scaled = all_data.copy()
all_data_scaled[feature_cols] = scaler_features.transform(all_data[feature_cols])
all_data_scaled[target_col] = scaler_target.transform(all_data[[target_col]])

# Función para generar pronósticos autorregresivos
def generate_forecasts(model, data, window_size, n_weeks_ahead, year):
    """
    Genera pronósticos autorregresivos para un año completo
    """
    model.eval()
    forecasts = []
    
    # Para cada barrio
    for barrio in data['id_bar'].unique():
        barrio_data = data[data['id_bar'] == barrio].sort_index()
        barrio_id = barrio_data['id_bar'].iloc[0]
        
        # Obtener última secuencia conocida
        last_sequence = barrio_data[feature_cols].iloc[-window_size:].values
        
        # Para mantener los valores de lluvia y temperatura del último año conocido
        # (asumiendo patrones estacionales)
        climate_pattern = barrio_data[feature_cols].iloc[-52:].values
        
        # Generar pronósticos semana a semana
        for week in range(1, n_weeks_ahead + 1):
            # Preparar entrada
            x = torch.FloatTensor(last_sequence).unsqueeze(0).to(device)
            
            # Hacer predicción
            with torch.no_grad():
                pred_scaled = model(x).cpu().numpy()[0, 0]
            
            # Desescalar predicción
            pred_original_scale = scaler_target.inverse_transform([[pred_scaled]])[0, 0]
            
            # Asegurar que la predicción no sea negativa
            pred_original_scale = max(0, pred_original_scale)
            
            # Guardar predicción
            forecasts.append({
                'id_bar': barrio_id,
                'anio': year,
                'semana': week,
                'dengue': pred_original_scale
            })
            
            # Actualizar secuencia para próxima predicción
            # Usar valores climáticos del año anterior (patrón estacional)
            climate_week_idx = (week - 1) % 52
            new_climate = climate_pattern[climate_week_idx] if climate_week_idx < len(climate_pattern) else last_sequence[-1]
            
            # Actualizar ventana deslizante
            last_sequence = np.vstack([last_sequence[1:], new_climate])
    
    return pd.DataFrame(forecasts)

# Generar pronósticos para 2022
forecasts_2022 = generate_forecasts(model_final, all_data_scaled, window_size, 52, 2022)

print(f"\nPronósticos generados: {len(forecasts_2022)} registros")
print("\nEstadísticas de pronósticos:")
print(forecasts_2022['dengue'].describe())

# ### 8. Creación del Archivo de Salida

# Crear DataFrame con formato requerido
output_df = pd.DataFrame()
output_df['id'] = (forecasts_2022['id_bar'].astype(str) + '_' + 
                   forecasts_2022['anio'].astype(str) + '_' + 
                   forecasts_2022['semana'].apply(lambda x: f"{x:02d}"))
output_df['dengue'] = forecasts_2022['dengue'].round(2)

# Verificar formato
print("\nPrimeras filas del archivo de salida:")
print(output_df.head(10))
print(f"\nTotal de registros: {len(output_df)}")

# Guardar archivo CSV
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_filename = f'pronosticos_GRU_simple_{timestamp}.csv'
output_df.to_csv(output_filename, index=False)
print(f"\nArchivo '{output_filename}' guardado exitosamente.")

# Visualización de algunos pronósticos
sample_barrios = forecasts_2022['id_bar'].unique()[:5]
fig = go.Figure()

for barrio in sample_barrios:
    barrio_forecast = forecasts_2022[forecasts_2022['id_bar'] == barrio]
    fig.add_trace(go.Scatter(
        x=barrio_forecast['semana'],
        y=barrio_forecast['dengue'],
        mode='lines+markers',
        name=f'Barrio {barrio}'
    ))

fig.update_layout(
    title='Pronósticos de Dengue para 2022 (Muestra de 5 Barrios)',
    xaxis_title='Semana del Año',
    yaxis_title='Casos de Dengue',
    hovermode='x unified'
)
fig.show()

print("\n### Proceso completado exitosamente ###")

[I 2025-06-26 23:21:42,094] A new study created in memory with name: no-name-6578b883-fe30-4163-9732-c9e1fc4187a2


Dispositivo de cómputo: cuda
Dimensiones del DataFrame: (3680, 20)

Primeras filas:
          id  id_bar  anio  semana  ESTRATO  area_barrio  dengue  \
0  4_2015_01       4  2015       1      3.0        0.560     0.0   
1  5_2015_01       5  2015       1      3.0        0.842     0.0   
2  3_2015_01       3  2015       1      1.0        0.781     0.0   
3  8_2015_01       8  2015       1      2.0        0.394     0.0   
4  9_2015_01       9  2015       1      2.0        0.292     0.0   

   concentraciones  vivienda  equipesado  sumideros  maquina  lluvia_mean  \
0              0.0       0.0         0.0        0.0      0.0     0.000651   
1              0.0       0.0         0.0        0.0      0.0     0.000651   
2              0.0       0.0         0.0        0.0      0.0     0.000651   
3              0.0       0.0         0.0        0.0      0.0     0.000651   
4              0.0       0.0         0.0        0.0      0.0     0.000651   

   lluvia_var  lluvia_max  lluvia_min  tempe

  0%|          | 0/40 [00:00<?, ?it/s]

[I 2025-06-26 23:21:48,633] Trial 0 finished with value: 0.13225137921316282 and parameters: {'window_size': 11, 'hidden_size': 64, 'num_layers': 2, 'dropout_rate': 0.3993643424746057, 'learning_rate': 0.00251874265241227, 'batch_size': 32}. Best is trial 0 with value: 0.13225137921316282.
[I 2025-06-26 23:21:59,075] Trial 1 finished with value: 0.13869592236975828 and parameters: {'window_size': 15, 'hidden_size': 32, 'num_layers': 2, 'dropout_rate': 0.3886802709708126, 'learning_rate': 0.00736017771881869, 'batch_size': 16}. Best is trial 0 with value: 0.13225137921316282.
[I 2025-06-26 23:22:09,347] Trial 2 finished with value: 0.10253322124481201 and parameters: {'window_size': 25, 'hidden_size': 32, 'num_layers': 1, 'dropout_rate': 0.4849393187735037, 'learning_rate': 0.00024246027653461173, 'batch_size': 64}. Best is trial 2 with value: 0.10253322124481201.
[I 2025-06-26 23:22:26,153] Trial 3 finished with value: 0.05757708922028541 and parameters: {'window_size': 46, 'hidden_siz


### Generando pronósticos para 2022 ###

Pronósticos generados: 520 registros

Estadísticas de pronósticos:
count    520.000000
mean       1.491658
std        0.185177
min        1.176948
25%        1.324188
50%        1.471453
75%        1.637254
max        1.937127
Name: dengue, dtype: float64

Primeras filas del archivo de salida:
          id  dengue
0  4_2022_01    1.44
1  4_2022_02    1.45
2  4_2022_03    1.49
3  4_2022_04    1.55
4  4_2022_05    1.94
5  4_2022_06    1.82
6  4_2022_07    1.82
7  4_2022_08    1.70
8  4_2022_09    1.70
9  4_2022_10    1.66

Total de registros: 520

Archivo 'pronosticos_GRU_simple_20250626_232555.csv' guardado exitosamente.



### Proceso completado exitosamente ###
